In [1]:
import pandas as pd
import numpy as np

In [176]:
data = pd.read_csv('data.csv', sep=',')
pd.set_option('display.max_rows', None)
data.pop('Lp.')
data

,Cena,Metraż,Odległość na uczelnię,Odległość do pracy
0,1100,19,3400,6700
1,900,10,7700,11600
2,700,9,5400,6100
3,1600,14,1600,5100
4,1000,11,4000,8600
5,1300,16,3200,800
6,1800,30,3400,6700
7,850,12,5500,6500
8,1450,11,4500,2400
9,900,9,4700,2000


In [3]:
def gain_type_concordance(a, b, p, q):
    # Współczynnik zgodności dla kryterium typu zysk
    if a - b >= -q:
        return 1
    elif a - b < -p:
        return 0
    else:
        return (p - (b - a))  / (p - q)
    
def cost_type_concordance(a, b, p, q):
    # Współczynnik zgodności dla kryterium typu koszt
    if a - b <= q:
        return 1
    elif a - b > p:
        return 0
    else:
        return (p - (a - b)) / (p - q)

def count_marginal_concordace(p, q):
    struct = np.zeros((data.shape[0], data.shape[0], data.shape[1]))

    for i, a in enumerate(data.values):
        for j, b in enumerate(data.values):
            if i == j:
                struct[i, j, :] = 1
                continue
            for k, g in enumerate(a):
                if k != 1:
                    struct[i, j, k] = gain_type_concordance(g, b[k], p[k], q[k])
                else:
                    struct[i, j, k] = cost_type_concordance(g, b[k], p[k], q[k])

    return struct

c = count_marginal_concordace([200, 4, 1000, 1000], [100, 2, 500, 500])

In [4]:
def gain_type_discordance(a, b, p, v):
    # Współczynnik niezgodności dla kryterium typu zysk
    if a - b <= -v:
        return 1
    elif a - b >= -p:
        return 0
    else:
        return ((b - a) - p)  / (v - p)
    

def cost_type_discordance(a, b, p, v):
    # Współczynnik niezgodności dla kryterium typu koszt
    if a - b >= v:
        return 1
    elif a - b <= p:
        return 0
    else:
        return (v - (a - b)) / (v - p)
    

def count_marginal_discordace(p, v, c):
    struct = np.zeros((data.shape[0], data.shape[0], data.shape[1]))

    for i, a in enumerate(data.values):
        for j, b in enumerate(data.values):
            if i == j:
                struct[i, j, :] = 0
                continue
            for k, g in enumerate(a):
                if k != 1:
                    struct[i, j, k] = gain_type_discordance(g, b[k], p[k], v[k]) if c[i, j, k] == 0 else 0
                else:
                    struct[i, j, k] = cost_type_discordance(g, b[k], p[k], v[k]) if c[i, j, k] == 0 else 0

    return struct
    
d = count_marginal_discordace([200, 2, 1000, 1000], [300, 6, 1500, 1500], c)

In [5]:
weights = [4, 4, 2, 3]

def count_total_concordance(c, w):
    return np.sum(c * w, axis=2) / np.sum(w)


C = count_total_concordance(c, weights)

In [6]:
def count_outranking_credibility(C, d):
    struct = np.zeros_like(C)
    for i in range(data.shape[0]):
        for j in range(data.shape[0]):
            f = np.where(d[i, j, :] > C[i, j])
            if len(f[0]) == 0:
                struct[i, j] = C[i, j]
            else:
                if np.any(d[i, j, f] == 1):
                    struct[i, j] = 0
                else:
                    struct[i, j] = C[i, j] * np.prod((1 - d[i, j, f]) / (1 - C[i, j]))
    return struct

R = count_outranking_credibility(C, d)

In [173]:
test_data = np.array([
    [1, 0.6, 0.3, 0.7, 0.55],
    [0, 1, 0.2, 0, 0],
    [0.58, 0.6, 1, 0.85, 0],
    [0.3, 0, 0.6, 1, 0.58],
    [0.6, 0, 0.6, 0.7, 1]
])

indices = np.arange(0, test_data.shape[0])
np.fill_diagonal(test_data, 0)
s = lambda x: -0.15 * x + 0.3
place = 1
ranking = {}



while test_data.size != 0:
    # 1, 2
    lambda_upper = np.max(test_data)

    # 4
    if lambda_upper == 0:
        break

    # 3
    lambda_lower = np.max(test_data[
        np.where(test_data < lambda_upper - s(lambda_upper))
    ])

    # 5
    checklist = np.where((test_data > lambda_lower) & (test_data > test_data.T + s(test_data)), test_data, 0)

    # 6
    strength = np.count_nonzero(checklist, axis=1)
    weakness = np.count_nonzero(checklist, axis=0)
    quality = strength - weakness

    best_alternatives = np.where(quality == np.max(quality))[0]
    idxs = indices[best_alternatives]
 
    # 8, 9
    while best_alternatives.size != 1:
        if np.all(checklist == 0):
            break

        initial_destilation = test_data[best_alternatives]
        initial_destilation = initial_destilation[:, best_alternatives]

        lambda_upper = lambda_lower
        lambda_lower = np.max(initial_destilation[
            np.where(initial_destilation < lambda_upper - s(lambda_upper))
        ])

        checklist = np.where((initial_destilation > lambda_lower) & (initial_destilation > initial_destilation.T + s(initial_destilation)), initial_destilation, 0)

        strength = np.count_nonzero(initial_destilation, axis=0)
        weakness = np.count_nonzero(initial_destilation, axis=1)
        quality = strength - weakness

        best_alternatives = np.where(quality == np.max(quality))[0]
        idxs = idxs[best_alternatives]

    # 7
    ranking[place] = idxs
    for i in idxs:
        indices = indices[indices != i]
    

    if indices.size != 0:
        for i in range(2):
            test_data = np.delete(test_data, ranking[place], axis=i)
    else:
        test_data = np.array([])
    place += 1

    

In [142]:
import numpy as np

# Przykładowa tablica ndarray
arr = np.array([[0, 0, 0],
                [0, 0, 0],
                [0, 0, 0]], dtype=float)
print(arr)
# Sprawdzenie czy tablica jest wypełniona zerami
is_zero = np.all(arr != 0)
print(is_zero)
if is_zero:
    print("Tablica jest wypełniona zerami.")
else:
    print("Tablica nie jest wypełniona zerami.")


[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
False
Tablica nie jest wypełniona zerami.


In [162]:
print(bool(np.array([])))

False


C:\Users\lukas\AppData\Local\Temp\ipykernel_4044\1403803955.py:1: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  print(bool(np.array([])))
